In [76]:
import pandas as pd



import datetime

In [138]:
global _WEEK_DICT, _HOLIDAY
_WEEK_DICT = {0:'ПН', 1:'ВТ', 2:'СР', 3:'ЧТ', 4:'ПТ', 5:'СБ', 6:'ВС'}
_HOLIDAY_MARKERS = [5,6]

In [ ]:
"""================================================================================================================================================="""
class CompatibleException(Exception):
    def __init__(self, message):
        self.message = message
        super().__init__(self.message)
    def __str__(self):
        return f"{self.message}"

"""================================================================================================================================================="""
class StrategyErrors:
    def __init__(self):
        self.message = None

    def UnCompatibleFilter(self, Strategy, Filter):
        self.message = f"Filter ({Filter.FilterIndex()}) not compatible with strategy ({Strategy.StrategyIndex()})"
        raise CompatibleException(self.message)

    def WrongIndicesType(self):
        self.message = f"Change DataFrame index to pandas datetime format"
        raise TypeError(self.message)

"""================================================================================================================================================="""
class StrategyHorizon:
    @classmethod
    def StrategyIndex(cls):
        return 'StrategyHorizon'

    def __init__(self):
        self._start = None
        self._end = None
        self.input_data = None

        self.filters = list()
        pass

    def input_params(self, start_horizon:datetime.timedelta, end_horizon: datetime.timedelta):
        self._start = start_horizon
        self._end = end_horizon

    def apply_filters(self, single_filter):
        if self.StrategyIndex not in single_filter.compatible:
            raise StrategyErrors().UnCompatibleFilter(Strategy=self, Filter=single_filter)
        self.filters.append(single_filter)

    def transfer_data(self, data: pd.DataFrame):
        if type(data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.input_data = data

    def _making_horizon(self):
        for dot_on_all_data in self.input_data.index:
            logical_filter = True
            #   SELECT BORDERS
            buffer_data = self.input_data.loc[dot_on_all_data + self._start: dot_on_all_data + self._end]
            for single_filter in self.filters:
                if not single_filter.apply_buffer_data(data=buffer_data).condition():
                    logical_filter = False
                    break

"""================================================================================================================================================="""
class CentralDotFilter:
    def __init__(self):
        self.compatible = [StrategyHorizon.StrategyIndex]
        self.buffer_data = None


    @classmethod
    def FilterIndex(cls):
        return 'CentralDotFilter'

    def apply_buffer_data(self, data: pd.DataFrame):
        if type(data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.buffer_data = data

    def condition(self, CentralDotCoords):
        pass

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""
class ConsistingTime:
    def __init__(self):
        self.compatible = [StrategyHorizon.StrategyIndex]
        self.buffer_data = None


    @classmethod
    def FilterIndex(cls):
        return 'ConsistingTime'

    def apply_buffer_data(self, data: pd.DataFrame):
        if type(data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.buffer_data = data

    def condition(self, *consist_time):
        for time in consist_time:
            if not time in self.buffer_data.index.time:
                return False

        return True

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""
class NoHolidays:
    def __init__(self):
        self.compatible = [StrategyHorizon.StrategyIndex]
        self.buffer_data = None


    @classmethod
    def FilterIndex(cls):
        return 'NoHolidays'

    def apply_buffer_data(self, data: pd.DataFrame):
        if type(data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.buffer_data = data

    def condition(self):
        for holiday in self.buffer_data.index.dayofweek:
            if holiday in _HOLIDAY_MARKERS:
                return False

        return True
"""================================================================================================================================================="""
#StrategyHorizon().apply_filters(CentralDotFilter())

In [ ]:
StrategyHorizon()

In [69]:
data = pd.read_csv('zip_data/AUDCAD.csv', index_col=0)
data.index = pd.to_datetime(data.index)

In [100]:
data.resample('1H').first().index.time

array([datetime.time(0, 0), datetime.time(1, 0), datetime.time(2, 0), ...,
       datetime.time(22, 0), datetime.time(23, 0), datetime.time(0, 0)],
      dtype=object)

In [70]:
hor = StrategyHorizon()
hor.input_params(datetime.timedelta(hours=-12))

pandas.core.indexes.datetimes.DatetimeIndex

In [91]:
(datetime.datetime(year=2020, month=12, day=1, hour=10, minute=20) - datetime.timedelta(hours=-12)).weekday()

1

In [103]:
d = data.resample('1H').first()

In [139]:
d.index.dayofweek[0] in _HOLIDAY_MARKERS

False

In [112]:
for _ in d.index[12:13]:
    print(d.loc[_+datetime.timedelta(hours=-12) : _+datetime.timedelta(hours=0)])

                         open      high       low     close
time                                                       
2018-11-01 00:00:00  0.931100  0.931135  0.930670  0.931025
2018-11-01 01:00:00  0.931275  0.931330  0.930935  0.931175
2018-11-01 02:00:00  0.931645  0.931725  0.931540  0.931690
2018-11-01 03:00:00  0.931975  0.932060  0.931935  0.931935
2018-11-01 04:00:00  0.934755  0.935005  0.934565  0.934570
2018-11-01 05:00:00  0.935190  0.935255  0.935075  0.935080
2018-11-01 06:00:00  0.935530  0.935580  0.935440  0.935445
2018-11-01 07:00:00  0.936920  0.936940  0.936800  0.936880
2018-11-01 08:00:00  0.936950  0.936950  0.936830  0.936865
2018-11-01 09:00:00  0.936520  0.936565  0.936505  0.936565
2018-11-01 10:00:00  0.937380  0.937535  0.937315  0.937515
2018-11-01 11:00:00  0.937930  0.937935  0.937760  0.937765
2018-11-01 12:00:00  0.937890  0.938245  0.937890  0.938180
2018-11-01 13:00:00  0.938350  0.938420  0.938195  0.938245
2018-11-01 14:00:00  0.938550  0.938610 

In [116]:
timesum = d.index.time[0]
result = [timesum]
for _ in d.index.time[1:]:
    timesum += _
    result.append(timesum)
result

TypeError: unsupported operand type(s) for +=: 'datetime.time' and 'datetime.time'

In [115]:
d.index.time

array([datetime.time(0, 0), datetime.time(1, 0), datetime.time(2, 0), ...,
       datetime.time(22, 0), datetime.time(23, 0), datetime.time(0, 0)],
      dtype=object)